In [ ]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#The code will check potential cascades of all possible pairs of medicines which are used the most often. This number represents, how many of them should be selected.
HOW_MANY_MOST_COMMON_DRUGS_TO_INVESTIGATE = 50

#csv file with columns name as following: mdc, atc, start_date, end_date
LOCATION_OF_PRESCRIBED_MEDICINES = "prescriptions_cleared.csv"

#csv files with columns named as following: atc_code, atc_name
ATC_CODES_PATH = "atccodes.csv"

In [ ]:
df = pd.read_csv(LOCATION_OF_PRESCRIBED_MEDICINES)

df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

drugs_for_investigation = df['atc'].value_counts().head(HOW_MANY_MOST_COMMON_DRUGS_TO_INVESTIGATE).index
drug_combinations = list(itertools.combinations(drugs_for_investigation, 2))

In [ ]:
atc_codes = pd.read_csv(ATC_CODES_PATH)
atc_codes.drop_duplicates(inplace=True)

In [ ]:
possible_cascades = pd.DataFrame(columns=['fixed_drug', 'potentionally_cascading', 'potentionally_cascading_prescribed_before_fixed', 'potentionally_cascading_prescribed_after_fixed', 'ratio', 'prescribed_same_day'])

df_filtered = df[df['atc'].isin(drugs_for_investigation)].copy()

for drug_a, drug_b  in drug_combinations:

    drug_A = df_filtered[df_filtered['atc'] == drug_a]
    drug_B = df_filtered[df_filtered['atc'] == drug_b]

    merged_A_B = pd.merge(drug_A, drug_B, on='mmd', suffixes=('_A', '_B'))

    A_before_B = (merged_A_B['start_date_A'] < merged_A_B['start_date_B']).sum()
    B_before_A = (merged_A_B['start_date_B'] < merged_A_B['start_date_A']).sum()
    same_date = (merged_A_B['start_date_B'] == merged_A_B['start_date_A']).sum()

    if B_before_A > 0 and A_before_B > 0: 
        new_row = {'fixed_drug': drug_a, 'potentionally_cascading': drug_b, 'potentionally_cascading_prescribed_before_fixed': B_before_A, 'potentionally_cascading_prescribed_after_fixed': A_before_B, 'ratio': A_before_B / (B_before_A), 'prescribed_same_day' : same_date}
        possible_cascades.loc[len(possible_cascades)] = new_row
        new_row = {'fixed_drug': drug_b, 'potentionally_cascading': drug_a, 'potentionally_cascading_prescribed_before_fixed': A_before_B, 'potentionally_cascading_prescribed_after_fixed': B_before_A, 'ratio': B_before_A / (A_before_B), 'prescribed_same_day' : same_date}
        possible_cascades.loc[len(possible_cascades)] = new_row


In [ ]:
#ADD NAMES TO DRUG CODES IN POTENTIONAL CASCADES

possible_cascades_with_names = pd.merge(possible_cascades, atc_codes, left_on="fixed_drug", right_on="atc_code", how="left")
possible_cascades_with_names.drop("atc_code", axis=1, inplace=True)
possible_cascades_with_names.rename(columns={"atc_name": "fixed_drug_name"}, inplace=True)

possible_cascades_with_names = pd.merge(possible_cascades_with_names, atc_codes, left_on="potentionally_cascading", right_on="atc_code", how="left")
possible_cascades_with_names.drop("atc_code", axis=1, inplace=True)
possible_cascades_with_names.rename(columns={"atc_name": "potentionally_cascading_name"}, inplace=True)

sorted_cascades = possible_cascades_with_names.sort_values(by="ratio", ascending=False)

## Cascades with highest ratio higer than 1, means that the potentionally cascading drug is prescribed more often after the prescription of a fixed drug. Higher the ratio, larger volume prescribed after fixed drug

# Visualizations

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def show_prescription_graph(fixed_drug, potentionally_cascading_graph, number_of_months=15):

    df['start_date'] = pd.to_datetime(df['start_date'])
    df['end_date'] = pd.to_datetime(df['end_date'])

    medicine_name_1 = atc_codes[atc_codes.atc_code == fixed_drug].iloc[0].iloc[1]
    medicine_name_2 = atc_codes[atc_codes.atc_code == potentionally_cascading_graph].iloc[0].iloc[1]

    df_medicine_1 = df[df['atc'] == fixed_drug]
    df_medicine_2 = df[df['atc'] == potentionally_cascading_graph]

    merged_df = pd.merge(df_medicine_1[['mmd', 'start_date']], 
                        df_medicine_2[['mmd', 'start_date']], 
                        on='mmd', 
                        suffixes=('_A', '_B'), how="inner")

    merged_df.drop_duplicates(inplace=True)
    merged_df['date_diff'] = -(merged_df['start_date_A'] - merged_df['start_date_B']).dt.days
    merged_df['month_diff'] = (merged_df['date_diff'] / 30).round().astype(int)
    merged_df = merged_df[(merged_df['month_diff'] <= number_of_months) & (merged_df['month_diff'] >= -number_of_months)]

    month_counts = merged_df['month_diff'].value_counts().sort_index()

    plt.figure(figsize=(10, 6))
    month_counts.plot(kind='bar', color='skyblue')
    plt.title(f'Months Difference Between Prescription of {medicine_name_1} and {medicine_name_2}. {medicine_name_1} is fixed')
    plt.xlabel('Months Before/After')
    plt.ylabel('Number of Patients')
    plt.xticks(rotation=0)
    plt.show()

## How to print these graphs - first parameter is fixed drug, second is potentional cascading, last number is number of months before and after prescription of fixed drug should be shown.

In [ ]:
show_prescription_graph('N02AC03', 'S01BC10', 3)